In [ ]:
from pathlib import Path
import datashader as ds
import datashader.transfer_functions as tf
import geopandas as gpd
import pandas as pd
import colorcet
from tqdm.auto import tqdm
from shapely.geometry import box

In [ ]:
parquet_files = list(Path("../scrape_amsa_vessel_positions/vessel_position_parquets").glob("*.parquet"))

In [ ]:
vessel_pos_gdf_list = [gpd.read_parquet(file) for file in tqdm(parquet_files)]

In [ ]:
vessel_pos_combined_gdf = pd.concat(vessel_pos_gdf_list, axis=0, ignore_index=True)

In [ ]:
vessel_pos_combined_gdf.to_parquet("ship_locations_full_history.parquet")

In [ ]:
min_lon = 110.0
max_lon = 157.0
min_lat = -45.0
max_lat = -5.0
australia_bbox = box(min_lon, min_lat, max_lon, max_lat)

In [ ]:
vessel_pos_combined_gdf = gpd.read_parquet("ship_locations_full_history.parquet")

In [ ]:
filtered_gdf = vessel_pos_combined_gdf[vessel_pos_combined_gdf.intersects(australia_bbox)]

In [ ]:
cvs = ds.Canvas(plot_width=2048, plot_height=1536)
agg = cvs.points(filtered_gdf, "LON", "LAT")
img = ds.tf.shade(agg, cmap=colorcet.kbc, how="log")
img = tf.set_background(img, "black")

In [ ]:
img.to_pil().save("ship_movement_heatmap_2048_1536_kbc.png")

In [ ]:
img.to_pil().save("ship_movement_heatmap_2048_1536_kbc.webp")